<img src="https://i.ibb.co/2dkgvqY/visualizing.png">
<img src="https://i.ibb.co/vw8Fvgz/Maya-Ayoubi-Font.png">
<center><a href="http://bit.ly/vizvax2021">bit.ly/vizvax2021</a></center>
<img src="https://www.gannett-cdn.com/presto/2020/03/07/USAT/97bc2b03-be50-4814-9389-9b5e52e3045b-promo-art.png?crop=5658,3183,x794,y621&width=1200" width="400" height="50" >

In [1]:
import pandas as pd
import plotly.express as px
import getpass
import os
import fbprophet
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import chart_studio

In [2]:
ACCESS_CODE = getpass.getpass('kaggle username: ')
SECRET_CODE = getpass.getpass('kaggle API token: ')

kaggle username: ········
kaggle API token: ········


In [3]:
os.environ['KAGGLE_USERNAME'] = ACCESS_CODE
os.environ['KAGGLE_KEY'] = SECRET_CODE

import kaggle as kg

In [4]:
kg.api.authenticate()
kg.api.dataset_download_files(dataset='gpreda/covid-world-vaccination-progress', path='gt.zip', unzip=True)
country_df = pd.read_csv('gt.zip/country_vaccinations.csv', encoding='ISO-8859-1')

kg.api.dataset_download_files(dataset='erikbruin/countries-of-the-world-iso-codes-and-population', path='gt.zip', unzip=True)
country_pop_df = pd.read_csv('gt.zip/countries_by_population_2019.csv', encoding='ISO-8859-1')

kg.api.dataset_download_files(dataset='erikbruin/countries-of-the-world-iso-codes-and-population', path='gt.zip', unzip=True)
iso_df = pd.read_csv('gt.zip/country_codes_2020.csv', encoding='ISO-8859-1')

In [5]:
states_url = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/us_state_vaccinations.csv'
us_states_df = pd.read_csv(states_url)

us_vaccinations_url = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/country_data/United%20States.csv'
us_vaccinations_df = pd.read_csv(us_vaccinations_url)

state_postal_url = 'https://raw.githubusercontent.com/ayoubimaya/COVID19-Vaccination-Progress/main/datasets/State_Postal_Codes.csv'
state_postal_df = pd.read_csv(state_postal_url)

In [6]:
username = 'mayoubi'
api_key = 'IeTvWckNVYH234JR9181' 

In [7]:
chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

<img src="https://i.ibb.co/fnprQrX/exploration.png">

In [8]:
country_df.head()

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,"BBIBP-CorV, Oxford/AstraZeneca, Pfizer/BioNTech",World Health Organization,https://covid19.who.int/
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,"BBIBP-CorV, Oxford/AstraZeneca, Pfizer/BioNTech",World Health Organization,https://covid19.who.int/
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,"BBIBP-CorV, Oxford/AstraZeneca, Pfizer/BioNTech",World Health Organization,https://covid19.who.int/
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,"BBIBP-CorV, Oxford/AstraZeneca, Pfizer/BioNTech",World Health Organization,https://covid19.who.int/
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,35.0,"BBIBP-CorV, Oxford/AstraZeneca, Pfizer/BioNTech",World Health Organization,https://covid19.who.int/


In [9]:
country_df.shape

(26552, 15)

In [10]:
country_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26552 entries, 0 to 26551
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   country                              26552 non-null  object 
 1   iso_code                             26552 non-null  object 
 2   date                                 26552 non-null  object 
 3   total_vaccinations                   14893 non-null  float64
 4   people_vaccinated                    14110 non-null  float64
 5   people_fully_vaccinated              11383 non-null  float64
 6   daily_vaccinations_raw               12298 non-null  float64
 7   daily_vaccinations                   26277 non-null  float64
 8   total_vaccinations_per_hundred       14893 non-null  float64
 9   people_vaccinated_per_hundred        14110 non-null  float64
 10  people_fully_vaccinated_per_hundred  11383 non-null  float64
 11  daily_vaccinations_per_milli

In [11]:
country_df.isna().sum()

country                                    0
iso_code                                   0
date                                       0
total_vaccinations                     11659
people_vaccinated                      12442
people_fully_vaccinated                15169
daily_vaccinations_raw                 14254
daily_vaccinations                       275
total_vaccinations_per_hundred         11659
people_vaccinated_per_hundred          12442
people_fully_vaccinated_per_hundred    15169
daily_vaccinations_per_million           275
vaccines                                   0
source_name                                0
source_website                             0
dtype: int64

In [12]:
us_states_df.shape

(10748, 14)

In [13]:
us_states_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10748 entries, 0 to 10747
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   date                                 10748 non-null  object 
 1   location                             10748 non-null  object 
 2   total_vaccinations                   10413 non-null  float64
 3   total_distributed                    10243 non-null  float64
 4   people_vaccinated                    10231 non-null  float64
 5   people_fully_vaccinated_per_hundred  9639 non-null   float64
 6   total_vaccinations_per_hundred       9741 non-null   float64
 7   people_fully_vaccinated              10149 non-null  float64
 8   people_vaccinated_per_hundred        9716 non-null   float64
 9   distributed_per_hundred              9728 non-null   float64
 10  daily_vaccinations_raw               10683 non-null  float64
 11  daily_vaccinations          

In [14]:
us_states_df.isna().sum()

date                                      0
location                                  0
total_vaccinations                      335
total_distributed                       505
people_vaccinated                       517
people_fully_vaccinated_per_hundred    1109
total_vaccinations_per_hundred         1007
people_fully_vaccinated                 599
people_vaccinated_per_hundred          1032
distributed_per_hundred                1020
daily_vaccinations_raw                   65
daily_vaccinations                       65
daily_vaccinations_per_million          756
share_doses_used                        505
dtype: int64

In [15]:
us_vaccinations_df.shape

(172, 7)

In [16]:
us_vaccinations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172 entries, 0 to 171
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   location                 172 non-null    object 
 1   date                     172 non-null    object 
 2   vaccine                  172 non-null    object 
 3   source_url               172 non-null    object 
 4   total_vaccinations       172 non-null    int64  
 5   people_vaccinated        171 non-null    float64
 6   people_fully_vaccinated  157 non-null    float64
dtypes: float64(2), int64(1), object(4)
memory usage: 9.5+ KB


In [17]:
us_vaccinations_df.isna().sum()

location                    0
date                        0
vaccine                     0
source_url                  0
total_vaccinations          0
people_vaccinated           1
people_fully_vaccinated    15
dtype: int64

In [18]:
state_postal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   state        51 non-null     object
 1   postal_code  51 non-null     object
dtypes: object(2)
memory usage: 944.0+ bytes


In [19]:
country_pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232 entries, 0 to 231
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Rank        232 non-null    int64  
 1   name        232 non-null    object 
 2   pop2019     232 non-null    float64
 3   pop2018     0 non-null      float64
 4   GrowthRate  232 non-null    float64
 5   area        232 non-null    float64
 6   Density     232 non-null    float64
dtypes: float64(5), int64(1), object(1)
memory usage: 12.8+ KB


In [20]:
iso_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232 entries, 0 to 231
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    232 non-null    object
 1   cca2    231 non-null    object
 2   cca3    232 non-null    object
 3   ccn3    232 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 7.4+ KB


In [21]:
country_limit = len(country_df) * .50
country_df = country_df.dropna(thresh=country_limit, axis=1)

#us_limit = len(country_df) * .50
#us_states_df = us_states_df.dropna(thresh=us_limit, axis=1)

In [22]:
nans = country_df[country_df['iso_code'].isnull()]
nans['country'].unique()

array([], dtype=object)

In [23]:
country_df['iso_code'].fillna('GBR', inplace = True)

In [24]:
'''every combo of vaccine'''

country_df['vaccines'].unique()

array(['BBIBP-CorV, Oxford/AstraZeneca, Pfizer/BioNTech',
       'Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, Sputnik V',
       'Oxford/AstraZeneca, Sputnik V',
       'Oxford/AstraZeneca, Pfizer/BioNTech', 'Oxford/AstraZeneca',
       'Oxford/AstraZeneca, Sinopharm/Beijing, Sputnik V',
       'Oxford/AstraZeneca, Sinovac, Sputnik V', 'Pfizer/BioNTech',
       'Johnson&Johnson, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',
       'Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V',
       'BBIBP-CorV, Sputnik V', 'Oxford/AstraZeneca, Sinopharm/Beijing',
       'Oxford/AstraZeneca, Sinovac', 'Moderna, Pfizer/BioNTech',
       'Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac',
       'Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac',
       'BBIBP-CorV, Oxford/AstraZeneca',
       'Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',
       'Covaxin, Oxford/AstraZeneca', 'BBIBP-CorV',
       'CanSino, Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac',
       'CanSino, Sinopharm/Bei

<img src="https://i.ibb.co/GWRgJcd/analysis.png">

<img src="https://i.ibb.co/0ZByV5r/mrna-title.png">
<img src="https://upload.wikimedia.org/wikipedia/commons/f/fe/Vaccine_candidate_mechanisms_for_SARS-CoV-2_%2849948301838%29.jpg" width="400" height="50" >

In [25]:
choropleth_grouping = country_df.groupby('country').max()

In [26]:
mrna_names = ['Moderna', 'Pfizer/BioNTech']
nonmrna_names = ['Oxford/AstraZeneca', 'Sinopharm/Beijing', 'Sinovac', 'RBD-Dimer', 
                 'Sputnik V', 'Sinopharm/Wuhan', 'Covaxin', 'Johnson&Johnson', 'EpiVacCorona',
                'Sinopharm/HayatVax', 'CanSino', 'Abdala', 'Soberana02', 'QazVac',  'Abdala', 
                 'Soberana02', 'BBIBP-CorV' ]
def mRNA(vaccine):
    has_mrna = False
    has_nonmrna = False
    
    for name in mrna_names:
        if name in vaccine:
            has_mrna = True
    for name in nonmrna_names:
        if name in vaccine:
            has_nonmrna = True
    
    if has_mrna and has_nonmrna:
        return 'Both'
    elif has_mrna:
        return 'mRNA'
    else:
        return 'Non-mRNA'
    
choropleth_grouping['type_of_vaccine'] = choropleth_grouping['vaccines'].apply(mRNA)

In [27]:
vacc_type_choro = px.choropleth(choropleth_grouping, 
                                locations='iso_code',
                                color='type_of_vaccine', 
                                color_discrete_sequence=px.colors.qualitative.G10, 
                                hover_name='type_of_vaccine')

vacc_type_choro.update_layout(title_text='Types of Vaccines Used', title_x=0.5)
vacc_type_choro.show()

In [28]:
'''You should comment out this line of code. This code is used create an HTML file for Plotly.'''

vacc_type_choro.write_html('/home/mayoubi/COVID_data/Images/vacc_type_choro_html.html')

### To preview plot: https://htmlpreview.github.io/?https://github.com/ayoubimaya/COVID19-Vaccination-Progress/blob/main/plots/vacc_type_choro_html.html

<img src="https://i.ibb.co/wCBWRfG/fastest-vaccines.png">

In [29]:
iso_pop_df = country_pop_df.merge(iso_df, left_on='name', right_on='name')
iso_pop_df = iso_pop_df[['name', 'cca3', 'pop2019']]
iso_pop_df['pop2019'] = iso_pop_df['pop2019'] * 1000
pd.options.display.float_format = "{:.2f}".format
iso_pop_df.head()

,name,cca3,pop2019
0,China,CHN,1433783686.00
1,India,IND,1366417754.00
2,United States,USA,329064917.00
3,Indonesia,IDN,270625568.00
4,Pakistan,PAK,216565318.00


In [30]:
country_df = country_df.merge(iso_pop_df, left_on='iso_code', right_on='cca3')

In [31]:
total_vacs_avg_df = country_df.groupby('iso_code').max()
vacs_per100_df = total_vacs_avg_df.sort_values(['total_vaccinations_per_hundred'], ascending=False)

'''filter for countries with populations greater than 10,000,0000'''
vacs_per100_df = vacs_per100_df[vacs_per100_df.pop2019 > 10_000_000].sort_values(['total_vaccinations_per_hundred'], ascending=False)
top_10_countries = vacs_per100_df[:10]

In [32]:
country_total_count = px.bar(top_10_countries, 
                             x=top_10_countries.index, 
                             y='total_vaccinations_per_hundred',
                             color='total_vaccinations_per_hundred', 
                             color_continuous_scale=px.colors.sequential.Emrld,
                             labels={
                                 'total_vaccinations_per_hundred': 'Total vaccinations (per hundred)',
                                 'location': ''
                             },)
country_total_count.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',})
country_total_count.update_coloraxes(showscale=False)
country_total_count.update_layout(title_text='Top 10 Countries (vaccinations per hundred)', title_x=0.5)
country_total_count.show()

In [33]:
'''You should comment out this line of code. This code is used create an HTML file for Plotly.'''

country_total_count.write_html('/home/mayoubi/COVID_data/Images/country_total_count_html.html')

### To preview: https://htmlpreview.github.io/?https://github.com/ayoubimaya/COVID19-Vaccination-Progress/blob/main/plots/country_total_count_html.html

In [34]:
daily_vacs_per_mil_df = country_df[['iso_code', 'country' ,'date', 'daily_vaccinations_per_million', 'pop2019']]
top_10_vacs = daily_vacs_per_mil_df.groupby('iso_code').max().sort_values(['daily_vaccinations_per_million'],ascending=False)

'''filter for countries with populations greater than 10,000,0000'''
top_10_vacs = top_10_vacs[top_10_vacs.pop2019 > 10_000_000].sort_values(['daily_vaccinations_per_million']
                                                                        ,ascending=False)[:10]
top_10_vacs

,country,date,daily_vaccinations_per_million,pop2019
iso_code,,,,
DOM,Dominican Republic,2021-06-22,16981.00,10738958.00
CUB,Cuba,2021-06-21,16429.00,11333483.00
CHL,Chile,2021-06-22,15614.00,18952038.00
TUR,Turkey,2021-06-23,14992.00,83429615.00
CHN,China,2021-06-23,14963.00,1433783686.00
KOR,South Korea,2021-06-23,14134.00,51225308.00
NLD,Netherlands,2021-06-23,13843.00,17097130.00
BEL,Belgium,2021-06-22,13280.00,11539328.00
CAN,Canada,2021-06-23,12134.00,37411047.00


In [35]:
top_10_countries = list(top_10_vacs.index)
countries_per_capita_df = daily_vacs_per_mil_df[daily_vacs_per_mil_df['iso_code'].isin(top_10_countries)]

In [36]:
daily_vacs_time_df = countries_per_capita_df.pivot_table(index='date', columns='iso_code', 
                                                       values='daily_vaccinations_per_million')
daily_vacs_time_df.reset_index(inplace=True)

In [37]:
daily_vacs_fig = px.line(daily_vacs_time_df, 
                         x='date', 
                         y=top_10_countries, 
                         color_discrete_sequence=px.colors.diverging.BrBG,
                         labels={
                             'date': ' ',
                             'value': ' '
                         })
daily_vacs_fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',})
daily_vacs_fig.update_layout(title_text='Daily Vaccinations (per million)', title_x=0.5)
daily_vacs_fig.show()

In [38]:
'''You should comment out this line of code. This code is used create an HTML file for Plotly.'''

daily_vacs_fig.write_html('/home/mayoubi/COVID_data/Images/daily_vacs_fig_html.html')

### To preview: https://htmlpreview.github.io/?https://github.com/ayoubimaya/COVID19-Vaccination-Progress/blob/main/plots/daily_vacs_fig_html.html

<img src="https://i.ibb.co/3cNGKVf/states.png">

In [39]:
states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York State', 'North Carolina', 'North Dakota', 'Ohio',
       'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island',
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
       'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin',
       'Wyoming']

In [40]:
us_states_df.head()

,date,location,total_vaccinations,total_distributed,people_vaccinated,people_fully_vaccinated_per_hundred,total_vaccinations_per_hundred,people_fully_vaccinated,people_vaccinated_per_hundred,distributed_per_hundred,daily_vaccinations_raw,daily_vaccinations,daily_vaccinations_per_million,share_doses_used
0,2021-01-12,Alabama,78134.00,377025.00,70861.00,0.15,1.59,7270.00,1.45,7.69,nan,nan,nan,0.21
1,2021-01-13,Alabama,84040.00,378975.00,74792.00,0.19,1.71,9245.00,1.53,7.73,5906.00,5906.00,1205.00,0.22
2,2021-01-14,Alabama,92300.00,435350.00,80480.00,nan,1.88,nan,1.64,8.88,8260.00,7083.00,1445.00,0.21
3,2021-01-15,Alabama,100567.00,444650.00,86956.00,0.28,2.05,13488.00,1.77,9.07,8267.00,7478.00,1525.00,0.23
4,2021-01-16,Alabama,nan,nan,nan,nan,nan,nan,nan,nan,7557.00,7498.00,1529.00,nan


In [41]:
us_states_df = us_states_df[us_states_df['location'].isin(states)]

In [42]:
us_states_df['location'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York State', 'North Carolina', 'North Dakota', 'Ohio',
       'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island',
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
       'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin',
       'Wyoming'], dtype=object)

In [43]:
us_states_choro = us_states_df.groupby('location').max()

In [44]:
us_states_choro = us_states_choro.reset_index()

In [45]:
def trim_all_columns(df):
    trim_strings = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(trim_strings)

state_postal_df = trim_all_columns(state_postal_df)

In [46]:
us_states_choro = us_states_choro.merge(state_postal_df, left_on='location', right_on='state')

In [47]:
states_choro = px.choropleth(us_states_choro, 
                             locations='postal_code', 
                             color='total_vaccinations_per_hundred',
                             hover_name='total_vaccinations_per_hundred', 
                             color_continuous_scale='Emrld',
                             locationmode='USA-states', scope='usa')

states_choro.update_layout(title_text='States Total Vaccination (per hundred)', title_x=0.5)
states_choro.show()

In [48]:
'''You should comment out this line of code. This code is used create an HTML file for Plotly.'''

states_choro.write_html('/home/mayoubi/COVID_data/Images/states_choro_html.html')

### To preview: https://htmlpreview.github.io/?https://github.com/ayoubimaya/COVID19-Vaccination-Progress/blob/main/plots/states_choro_html.html

In [49]:
us_total_fig = px.line(us_vaccinations_df, 
                       x='date', 
                       y='total_vaccinations',
                       labels={
                           'total_vaccinations': '# of total vaccinations',
                            'date': ' '
                       })
us_total_fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',})
us_total_fig.update_layout(showlegend=False)
us_total_fig.update_layout(title_text='US Total Vaccinations', title_x=0.5)
us_total_fig.show()

In [50]:
'''You should comment out this line of code. This code is used create an HTML file for Plotly.'''

us_total_fig.write_html('/home/mayoubi/COVID_data/Images/us_total_fig_html.html')

### To preview: https://htmlpreview.github.io/?https://github.com/ayoubimaya/COVID19-Vaccination-Progress/blob/main/plots/us_total_fig_html.html

<img src="https://i.ibb.co/9tQPW60/100-days.png">
<img src="https://thumbs.gfycat.com/LiquidQuestionableGrassspider-max-1mb.gif"></center>
<br> <center> On January 25th, President Joe Biden pledged 150 million vaccinations in 100 days. </center>

In [51]:
us_vaccinations_df['daily_vaccinations'] = us_vaccinations_df['total_vaccinations'] - us_vaccinations_df['total_vaccinations'].shift(1)

In [52]:
'''Fit the model by instantiating a new Prophet object. Using logistics settings to the forecasting procedure.
Call its fit method and pass in the historical dataframe.'''

total_vacc = us_vaccinations_df.rename(columns={'date': 'ds', 'total_vaccinations': 'y'})
total_vacc['cap'] = 328000000
m = Prophet(growth='logistic')
m.fit(total_vacc)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [53]:
'''ds column contains dates for which a prediction is to be made. You can get a suitable dataframe that 
extends into the future a specified number of days using the helper method Prophet.make_future_dataframe.'''

future_total_vacc = m.make_future_dataframe(periods=71)
future_total_vacc['cap'] = 328000000
print(future_total_vacc.tail())

            ds        cap
238 2021-08-30  328000000
239 2021-08-31  328000000
240 2021-09-01  328000000
241 2021-09-02  328000000
242 2021-09-03  328000000


In [54]:
'''The predict method will assign each row in future a predicted value which it names yhat. The forecast object 
here is a new dataframe that includes a column yhat with the forecast, as well as columns for components and 
uncertainty intervals.'''

forecast_total_vacc = m.predict(future_total_vacc)
print(forecast_total_vacc[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail())

            ds         yhat   yhat_lower   yhat_upper
238 2021-08-30 327069674.79 324460084.52 329635421.21
239 2021-08-31 326913987.45 324186054.51 329602690.25
240 2021-09-01 326464572.08 323929955.47 329043822.47
241 2021-09-02 326994868.08 324550834.79 329308775.09
242 2021-09-03 327219588.09 324760316.94 329528521.15


In [55]:
total_vacc_pred_fig = plot_plotly(m, forecast_total_vacc)
total_vacc_pred_fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)',})
total_vacc_pred_fig.update_layout(title_text='Prediction of Total US Vaccinations', title_x=0.5)
total_vacc_pred_fig.show()

In [56]:
'''You should comment out this line of code. This code is used create an HTML file for Plotly.'''

total_vacc_pred_fig.write_html('/home/mayoubi/COVID_data/Images/total_vacc_pred_fig_html.html')

### To preview: https://htmlpreview.github.io/?https://github.com/ayoubimaya/COVID19-Vaccination-Progress/blob/main/plots/total_vacc_pred_fig_html.html

<img src="https://i.ibb.co/x3b4wcK/reach-out.png">
<center><a href="http://bit.ly/vizvax2021">bit.ly/vizvax2021</a></center>
<img src="https://i.ibb.co/mBNMpHY/contact-info.png">